In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline

import matplotlib.style as style
style.use('seaborn-whitegrid')

import os
import pprint
# import googlemaps
# import time
import pickle
from random import randint
from collections import defaultdict

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, KFold
from sklearn.linear_model import RidgeCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

In [78]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.naive_bayes import MultinomialNB

import nltk
from nltk.tokenize import word_tokenize, sent_tokenize, TreebankWordTokenizer, wordpunct_tokenize
from nltk.tag import pos_tag
from nltk.chunk import ne_chunk
from nltk.util import ngrams
from nltk.corpus import stopwords

from textblob import TextBlob

import string



In [176]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

In [3]:
# Set pandas options
pd.set_option('max_rows', 10)
# pd.set_option('max_colwidth', -1)
# pd.set_option('display.width', 150)
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)

In [13]:
df = pd.read_csv('data/listing_descriptions.csv', index_col=0)

In [103]:
summaries[0]

"Located on Williams bike corridor, some of Portland's best restaurants, bars and coffee shops are within a 5min drive and downtown is only 10min away.  Also features a wrap around patio with a covered awning to stay dry during the rainy season."

In [77]:
df.head(2)

id  host_id                                            summary  \
0   7893    21489  Located on Williams bike corridor, some of Por...   
1  12899    49682  Settle in to a space of your own. You'll have ...   

                                               space  \
0  This NE Portland 2 bedroom home is located jus...   
1  We live in a neighborhood of beautiful gardens...   

                                         description  \
0  Located on Williams bike corridor, some of Por...   
1  Settle in to a space of your own. You'll have ...   

                               neighborhood_overview  \
0                                                NaN   
1  We're within walking distance of a grocery and...   

                                               notes  \
0                                                NaN   
1  We welcome musicians and even have a guitar fo...   

                                          host_about  
0  I love to cook, sail and play the mandolin ver...  
1  We enjoy cooking, singing, hiking, bicycles, t...

In [81]:
df.columns

Index(['id', 'host_id', 'summary', 'space', 'description',
       'neighborhood_overview', 'notes', 'host_about'],
      dtype='object')

In [92]:
# Fill NaNs with empty strings
for col in ['summary', 'space', 'description', 'neighborhood_overview', 'notes', 'host_about']:
    df[col].fillna('', inplace=True)


# Make sure there are not any null values remaining
df.head(2)

id  host_id                                            summary  \
0   7893    21489  Located on Williams bike corridor, some of Por...   
1  12899    49682  Settle in to a space of your own. You'll have ...   

                                               space  \
0  This NE Portland 2 bedroom home is located jus...   
1  We live in a neighborhood of beautiful gardens...   

                                         description  \
0  Located on Williams bike corridor, some of Por...   
1  Settle in to a space of your own. You'll have ...   

                               neighborhood_overview  \
0                                                      
1  We're within walking distance of a grocery and...   

                                               notes  \
0                                                      
1  We welcome musicians and even have a guitar fo...   

                                          host_about  
0  I love to cook, sail and play the mandolin ver...  
1  We enjoy cooking, singing, hiking, bicycles, t...

In [244]:
descriptions = df['description']
about_host_text = df['host_about']
summaries = df['summary']
type(summaries)

pandas.core.series.Series

#### Remove punctuation, make lowercase, etc.

In [163]:
# Make lower case
summaries = summaries.apply(lambda x: x.lower())



In [ ]:
# for text in texts:
#     sentences = nltk.sent_tokenize(text)
#     for sentence in sentences:
#         words = nltk.word_tokenize(sentence)
#         tagged_words = nltk.pos_tag(words)
#         ne_tagged_words = nltk.ne_chunk(tagged_words)
#         print ne_tagged_words

In [164]:
summaries = pd.DataFrame(summaries)

In [165]:
# Remove punctuation
summaries['nopunc'] = summaries['summary'].str.replace('[^\w\s]','')

In [166]:
summaries

summary  \
0     located on williams bike corridor, some of por...   
1     settle in to a space of your own. you'll have ...   
2     newly remodeled 2 bedroom (3 queen beds), 2 fu...   
3     if you are an urban farming pioneer, enthusias...   
4     if you are an urban farming pioneer, enthusias...   
...                                                 ...   
5400  moderate sized room with north garden and east...   
5401  modern portland home with mid-century charm! t...   
5402  1 bed 1 bath apartment living in ultra posh do...   
5403  this is a cozy basement studio nestled on the ...   
5404  cozy camper space with access to a hip house a...   

                                                 nopunc  
0     located on williams bike corridor some of port...  
1     settle in to a space of your own youll have 2 ...  
2     newly remodeled 2 bedroom 3 queen beds 2 full ...  
3     if you are an urban farming pioneer enthusiast...  
4     if you are an urban farming pioneer enthusiast...  
...                                                 ...  
5400  moderate sized room with north garden and east...  
5401  modern portland home with midcentury charm thi...  
5402  1 bed 1 bath apartment living in ultra posh do...  
5403  this is a cozy basement studio nestled on the ...  
5404  cozy camper space with access to a hip house a...  

[5405 rows x 2 columns]

In [167]:
# Tokenize by sentence
summaries['sent_tokenize'] = summaries['nopunc'].apply(lambda x: sent_tokenize(x))

In [153]:
# Tokenize by word
tokenizer = TreebankWordTokenizer()
summaries['word_tok'] = summaries['sent_tok'].apply(lambda x: [tokenizer.tokenize(y) for y in x])

In [155]:
summaries['no

summary  \
0     located on williams bike corridor, some of por...   
1     settle in to a space of your own. you'll have ...   
2     newly remodeled 2 bedroom (3 queen beds), 2 fu...   
3     if you are an urban farming pioneer, enthusias...   
4     if you are an urban farming pioneer, enthusias...   
...                                                 ...   
5400  moderate sized room with north garden and east...   
5401  modern portland home with mid-century charm! t...   
5402  1 bed 1 bath apartment living in ultra posh do...   
5403  this is a cozy basement studio nestled on the ...   
5404  cozy camper space with access to a hip house a...   

                                                 nopunc  \
0     located on williams bike corridor some of port...   
1     settle in to a space of your own youll have 2 ...   
2     newly remodeled 2 bedroom 3 queen beds 2 full ...   
3     if you are an urban farming pioneer enthusiast...   
4     if you are an urban farming pioneer enthusiast...   
...                                                 ...   
5400  moderate sized room with north garden and east...   
5401  modern portland home with midcentury charm thi...   
5402  1 bed 1 bath apartment living in ultra posh do...   
5403  this is a cozy basement studio nestled on the ...   
5404  cozy camper space with access to a hip house a...   

                                                no_punc  \
0     [located on williams bike corridor some of por...   
1     [settle in to a space of your own youll have 2...   
2     [newly remodeled 2 bedroom 3 queen beds 2 full...   
3     [if you are an urban farming pioneer enthusias...   
4     [if you are an urban farming pioneer enthusias...   
...                                                 ...   
5400  [moderate sized room with north garden and eas...   
5401  [modern portland home with midcentury charm th...   
5402  [1 bed 1 bath apartment living in ultra posh d...   
5403  [this is a cozy basement studio nestled on the...   
5404  [cozy camper space with access to a hip house ...   

                                               word_tok  
0     [[located, on, williams, bike, corridor, some,...  
1     [[settle, in, to, a, space, of, your, own, you...  
2     [[newly, remodeled, 2, bedroom, 3, queen, beds...  
3     [[if, you, are, an, urban, farming, pioneer, e...  
4     [[if, you, are, an, urban, farming, pioneer, e...  
...                                                 ...  
5400  [[moderate, sized, room, with, north, garden, ...  
5401  [[modern, portland, home, with, midcentury, ch...  
5402  [[1, bed, 1, bath, apartment, living, in, ultr...  
5403  [[this, is, a, cozy, basement, studio, nestled...  
5404  [[cozy, camper, space, with, access, to, a, hi...  

[5405 rows x 4 columns]

In [221]:
tf_vectorizer = CountVectorizer(max_df=0.5,
                                min_df=2,
                                lowercase=True,
                                strip_accents='unicode',
                                token_pattern = r'\b[a-zA-Z]{3,}\b',
                                stop_words='english', ngram_range=(1,2)
                               )
dtm_tf = tf_vectorizer.fit_transform(summaries)

In [207]:
dtm_tf.shape

(5405, 4706)

In [208]:
# for TF DTM
lda_tf = LatentDirichletAllocation(n_topics=10, random_state=0)
lda_tf.fit(dtm_tf)

# # for TFIDF DTM
# lda_tfidf = LatentDirichletAllocation(n_topics=20, random_state=0)
# lda_tfidf.fit(dtm_tfidf)

/Users/brenner/anaconda3/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:314: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='batch', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=10, n_jobs=None, n_topics=10, perp_tol=0.1,
             random_state=0, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [209]:
pyLDAvis.sklearn.prepare(lda, dtm_tf, tf_vectorizer)

AssertionError: Topic-term distributions and document-term matrix have different number of columns, 1741 != 4706.

In [62]:
tf.shape

(5405, 5058)

In [66]:
vectorizer = TfidfVectorizer(stop_words="english", ngram_range=(1,2))
doc_vectors = vectorizer.fit_transform(summaries)

In [70]:
doc_vectors

<5405x89207 sparse matrix of type '<class 'numpy.float64'>'
	with 359635 stored elements in Compressed Sparse Row format>

In [201]:
lda_tf = LatentDirichletAllocation(n_components=10, max_iter=5,
                                learning_method='online',
                                learning_offset=10.0, batch_size=128,
                                random_state=49, evaluate_every=-1)
lda_tf.fit(tf)

In [220]:
vocab = tf_vectorizer.get_feature_names()

for topic in range(10):
    print(f"TOPIC {topic}")
    for j in np.argsort(-lda.components_,1)[topic,:10]:
        print(vocab[j])
    print()

TOPIC 0
place
great
business
travelers
close
couples
good
solo
adventurers
neighborhood

TOPIC 1
tabor
fun
outdoor
covered
backyard
scene
patio
fred
providence
serene

TOPIC 2
district
pearl
score
ceilings
loft
high
heart
steps
stay
bar

TOPIC 3
new
brand
experience
seasons
home
modern
designed
visit
school
guesthouse

TOPIC 4
north
friendly
people
welcome
live
kenton
light
friends
avenue
pets

TOPIC 5
room
private
bed
bedroom
queen
bathroom
kitchen
living
bath
space

TOPIC 6
coffee
fridge
tea
microwave
breakfast
mini
available
access
maker
use

TOPIC 7
alberta
apartment
arts
district
basement
located
house
home
blocks
studio

TOPIC 8
dryer
washer
house
tub
kitchen
unit
hot
enjoy
space
cozy

TOPIC 9
cute
square
sleeping
feet
nook
smoking
transport
backyard
cottage
inside



In [258]:
def run_lda_with_defaults(corpus, num_topics):
    tf_vectorizer = CountVectorizer(lowercase=True,
                                    strip_accents='unicode',
                                    token_pattern = r'\b[a-zA-Z]{3,}\b',
                                    stop_words='english')
    
    dtm_tf = tf_vectorizer.fit_transform(corpus)
    
    lda_tf = LatentDirichletAllocation(n_components=num_topics, random_state=0, n_jobs=-1)
    lda_tf.fit(dtm_tf)
    
    return pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer)

In [256]:
def run_lda(corpus, num_topics):
    tf_vectorizer = CountVectorizer(max_df=0.5,
                                    min_df=2,
                                    lowercase=True,
                                    strip_accents='unicode',
                                    token_pattern = r'\b[a-zA-Z]{3,}\b',
                                    stop_words='english',
                                    ngram_range=(1,3))
    
    dtm_tf = tf_vectorizer.fit_transform(corpus)
    
    lda_tf = LatentDirichletAllocation(n_components=num_topics, random_state=0, learning_method='online', n_jobs=-1)
    lda_tf.fit(dtm_tf)
    
    return pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer)

In [255]:
def run_tfidf_lda(corpus, num_topics):
    tfidf_vectorizer = TfidfVectorizer(max_df=0.5,
                                    min_df=2,
                                    lowercase=True,
                                    strip_accents='unicode',
                                    token_pattern = r'\b[a-zA-Z]{3,}\b',
                                    stop_words='english')
    
    dtm_tfidf = tfidf_vectorizer.fit_transform(corpus)
    
    lda_tfidf = LatentDirichletAllocation(n_components=num_topics, random_state=0, learning_method='online', n_jobs=-1)
    lda_tfidf.fit(dtm_tfidf)
    
    return pyLDAvis.sklearn.prepare(lda_tfidf, dtm_tfidf, tfidf_vectorizer)

In [222]:
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                stop_words = 'english',
                                lowercase = True,
                                token_pattern = r'\b[a-zA-Z]{3,}\b',
                                max_df = 0.5, 
                                min_df = 10, ngram_range=(1,2))
dtm_tf = tf_vectorizer.fit_transform(summaries)
print(dtm_tf.shape)

(5405, 3766)


In [223]:
lda_tf = LatentDirichletAllocation(n_topics=10, random_state=0, learning_method='online', batch_size=1028)
lda_tf.fit(dtm_tf)

/Users/brenner/anaconda3/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:314: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


LatentDirichletAllocation(batch_size=1028, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=10, n_jobs=None, n_topics=10, perp_tol=0.1,
             random_state=0, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [ ]:
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer)

In [259]:
run_lda_with_defaults(descriptions, 10)

/Users/brenner/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5      0.069935  0.017552       1        1  24.356309
1      0.039434  0.069310       2        1  17.605827
0     -0.084185  0.154308       3        1  17.480405
6     -0.067826  0.070179       4        1  11.803181
9      0.036567  0.047240       5        1   9.663327
3      0.060626 -0.063598       6        1   6.518400
7      0.093458 -0.041467       7        1   4.518141
4     -0.000710  0.001390       8        1   3.071818
2     -0.252597 -0.130306       9        1   2.701773
8      0.105296 -0.124607      10        1   2.280819, topic_info=      Category         Freq         Term        Total  loglift  logprob
term                                                                   
9958   Default  5656.000000         room  5656.000000  30.0000  30.0000
486    Default  2556.000000    apartment  2556.000000  29.0000  29.0000
8338   Default  1774.000000         park  1774.000000  28.0000  28.0000
8870   Default  6697.000000     portland  6697.000000  27.0000  27.0000
1041   Default  3488.000000          bed  3488.000000  26.0000  26.0000
...        ...          ...          ...          ...      ...      ...
977    Topic10    53.709214     bathroom  2863.712166  -0.1957  -5.2560
5075   Topic10    51.609911        great  2288.945358  -0.0115  -5.2958
2384   Topic10    45.134626  comfortable  1644.301980   0.1852  -5.4299
10914  Topic10    42.105588     spacious  1079.955990   0.5361  -5.4994
7722   Topic10    42.159485         need  1299.012425   0.3527  -5.4981

[854 rows x 6 columns], token_table=       Topic      Freq        Term
term                              
23         1  0.046196  absolutely
23         2  0.027717  absolutely
23         4  0.083152  absolutely
23         6  0.674455  absolutely
23         8  0.129347  absolutely
...      ...       ...         ...
13157      8  0.898537        yurt
13164      2  0.061413         zen
13164      5  0.153533         zen
13164      7  0.706251         zen
13164      8  0.092120         zen

[2766 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 2, 1, 7, 10, 4, 8, 5, 3, 9])